# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

In [145]:
# !pip install scikit-learn==0.20.0 --upgrade
# !pip install imbalanced-learn==0.4.3

In [146]:
import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
import seaborn as sns

In [147]:
df_data_1 = pd.read_csv("C://Users/hamil\Dropbox/Estudos/MaratonaBehindTheCodeIbm/desafio-2-2020-master/Assets/Data/dataset_desafio_2.csv")
df_data_1.head()


,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


In [148]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from imblearn.over_sampling import SMOTE, ADASYN

# All sklearn Transforms must have the `transform` and `fit` methods
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # Retornamos um novo dataframe sem as colunas indesejadas
        return data.drop(labels=self.columns, axis='columns')
    
#Preenche o campo indicado (coluna) com o valor desejado    
class FillWith(BaseEstimator, TransformerMixin):
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        #faz a inclusão do valor recebido quando o campo for NaN
        data[self.column] = data[self.column].fillna(self.value)  # insere valor recebido quando não tem a informação
        # Retornamos um novo dataframe
        return data     
    

    
#cria uma coluna com média geral das notas
class MediaGeral(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #def AvaliaCaso(faltoso, nota_de, nota_em, nota_mf, nota_go):
        def AvaliaCaso(nota_de, nota_em, nota_mf, nota_go):            
            conta = 0
            valores = 0
            if nota_de > 0 and not pd.isna(nota_de):
                conta += 1
                valores += nota_de
            if nota_em > 0 and not pd.isna(nota_em):
                conta += 1
                valores += nota_em
            if nota_mf > 0 and not pd.isna(nota_mf):
                conta += 1
                valores += nota_mf
            if nota_go > 0 and not pd.isna(nota_go):
                conta += 1
                valores += nota_go
            if conta > 0:    
                media = valores / conta
            else:
                media = 0
            return media
        
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        #faz a criação da coluna "MEDIA_GERAL" sem considerar 0 para não faltosos
        #data['MEDIA_GERAL'] = data.apply(lambda x: AvaliaCaso(x.FALTOSO, x.NOTA_DE, x.NOTA_EM, x.NOTA_MF, x.NOTA_GO), axis=1)
        data['MEDIA_GERAL'] = data.apply(lambda x: AvaliaCaso(x.NOTA_DE, x.NOTA_EM, x.NOTA_MF, x.NOTA_GO), axis=1)
        # Retornamos um novo dataframe
        return data     
    
    
#Altera nota quando está NaN ou 0 e limita a nota em 10
class NotaUnica(BaseEstimator, TransformerMixin):
    def __init__(self, qual):
        self.qual = qual
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #def AvaliaNota(faltoso, nota_media, qual, nota):
        def AvaliaNota(nota_media, qual, nota):    
            #verifica se nota é 0 ou NaN
            if pd.isna(nota) or nota == 0:
                if nota_media > 0:
                    nota = 10
                else:
                    nota = 0
            elif nota > 10:    
              nota = 10
            return nota
        
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        #faz verificação se a nota é 0 ou NaN e coloca novo valor
        #data[self.qual] = data.apply(lambda x: AvaliaNota(x.FALTOSO, x.MEDIA_GERAL, self.qual, x[self.qual]), axis=1)
        data[self.qual] = data.apply(lambda x: AvaliaNota(x.MEDIA_GERAL, self.qual, x[self.qual]), axis=1)
        # Retornamos um novo dataframe
        return data       
    
    
#cria uma coluna com TRUE para os TUDO ZERO e FALSE para os demais    
class Dificuldade(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        print("DIFICULDADE")
        print(len(X))
        print(len(y))
        return self
    
    def transform(self, X):
        print('dificuldade', X.head())
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        #faz a criação da coluna "DIFICULDADE" 
        data['DIFICULDADE'] = data.apply(lambda x: True if (x['MEDIA_GERAL'] == 0) else False, axis=1)
        # Retornamos um novo dataframe
        return data     
    
    
#transforma um campo em determinado tipo
class TrocaTipo(BaseEstimator, TransformerMixin):
    def __init__(self, coluna, tipo):
        self.coluna = coluna
        self.tipo = tipo
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # faz a transformação do tipo da coluna
        data[self.coluna] = data[self.coluna].astype(self.tipo) 
        # Retornamos um novo dataframe
        return data 
    
    
#executa o SMOTE
class ExecutaSmote(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        print(self)
        X_apoio = X.copy()
        y_apoio = y.copy()
        
        X, y = SMOTE().fit_sample(X_apoio, y_apoio)
        print("DEPOIS*****************")
        print(len(X))
        print(len(y))
        retorno = (X, y)
        return retorno

    def transform(self, X):
        return self       

In [149]:
# Criação de uma Transform personalizada ``DropColumns``

rm_columns = DropColumns(
    columns=["NOME", "MATRICULA"]
)

In [150]:
#Campo Inglês
# Instanciando um ajuste FillWOne
fillwith = FillWith(
    column="INGLES",  # Essa transformação recebe como parâmetro o nome da coluna a tratar,
    value=1 #preenche com 1 (não sabe inglês) quando vazio
)

In [151]:
#depois de removidos os NaN transforma coluna INGLES em Inteiro
#df_data_2.INGLES = df_data_2.INGLES.astype('int64') 

trocatipo = TrocaTipo(
    coluna = "INGLES",
    tipo = "int64"
)

In [152]:
#Ajustando nota zero ou NaN individualmente
mediageral = MediaGeral()

In [153]:
# Ajustando nota zerada DE
notaUnica_em = NotaUnica(
    qual = "NOTA_EM"
)

In [154]:
# Ajustando nota zerada DE
notaUnica_de = NotaUnica(
    qual = "NOTA_DE"
)

In [155]:
# Ajustando nota zerada DE
notaUnica_mf = NotaUnica(
    qual = "NOTA_MF"
)

In [156]:
# Ajustando nota zerada DE
notaUnica_go = NotaUnica(
    qual = "NOTA_GO"
)

In [157]:
# Aplicando a criação do campo ``Dificuldade`` ao conjunto de dados base
dificuldade = Dificuldade()

In [158]:
# Criação de um objeto ``SimpleImputer``

si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=0,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [159]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_1[features]
y = df_data_1[target]

In [160]:
# Aplica o SMOTE no dataset
executasmote = ExecutaSmote()

In [161]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=337)

In [166]:
len(X_train)

16000

In [167]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('fillwith', fillwith),
        ('mediageral', mediageral),
        ('notaUnica_em', notaUnica_em),
        ('notaUnica_de', notaUnica_de),
        ('notaUnica_mf', notaUnica_mf),
        ('notaUnica_go', notaUnica_go),
        ('executasmote', executasmote),
        ('dificuldade', dificuldade),
        ('imputer', si),
        ('dtc', RandomForestClassifier(criterion='gini', max_depth= 40, max_features='auto', n_estimators= 262)),
    ]
)

In [168]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train.values.ravel())

ExecutaSmote()
DEPOIS*****************
33005
33005
DIFICULDADE
2
16000


AttributeError: 'tuple' object has no attribute 'head'

In [ ]:
y_pred = my_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 4)))